Working jupyter to create function to create a list of aminoacids likely to be used for mutation

In [2]:
# import packages
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt


In [152]:
def pdb2df(input_path, pdb_file, output_path, df=None):
    from function import pdb2pqr
    if os.path.isfile(os.path.join(output_path, f'{(pdb_file.split(".")[0]).split("-")[1]}.pqr')):
        print('PQR file already exists')
    pdb2pqr(input_path, output_path, pdb_file)
    
    Atom_col = ['atom_num','atom_name', 'aa_name', 'aa_num', 'x', 'y', 'z', 'charge', 'radius']
    Atom_array = np.empty((0, 9))
    name = f'{(pdb_file.split('.')[0]).split('-')[1]}.pqr'

    with open(os.path.join(output_path, str(name))) as f:
        for line in f:
            line = line.replace('-', '  -')
            if line.startswith('ATOM'):
                line_array = np.array([[int(line.split()[1]), str(line.split()[2]), str(line.split()[3]), int(line.split()[4]), float(line.split()[5]), float(line.split()[6]), float(line.split()[7]), float(line.split()[8]), float(line.split()[9])]]) 
                Atom_array = np.append(Atom_array, line_array, axis = 0)
    
    
    if df is not None:
        Atom_df = pd.DataFrame(Atom_array, columns=Atom_col)
        return Atom_df

    return Atom_df
                


In [151]:
input_path = './data/pdbs'
pdb_file = 'AF-C0H3V8-F1.pdb'
output_path = './data/test'

test = pdb2df(input_path, pdb_file, output_path)

FileNotFoundError: [Errno 2] No such file or directory: './data/test\\C0H3V8.pqr'

In [147]:
def aa_mut_select(input_path, pdb_file, output_path, df=None): ##add delete entrys with nan (see distance function)
    from function import salt_bridge
    from function import H_bond_calc
    from function import VdW_interaction
    pqr_file = f'{(pdb_file.split('.')[0]).split('-')[1]}.pqr'
    if df is not None:
        print('1')
        Prot_df = pdb2df(input_path, pdb_file, output_path, df=True)
        print('Prot_df finished')
    else:
        print('2')
        Prot_df = pdb2df(input_path, pdb_file, output_path)
        print('Prot_arr finished')

    Salt_bridge = salt_bridge(input_path, pdb_file)
    print('Salt_bridge finished')
    H_bond = H_bond_calc(output_path, pqr_file)
    print('H_bond finished')
    VdW_clust, VdW_vol = VdW_interaction(input_path, pdb_file)
    print('VdW_interaction finished')
    
    return Salt_bridge, H_bond, VdW_clust, VdW_vol, Prot_df   #, Prot_df
    
    
    

In [154]:
input_path = './data/pdbs'
pdb_file = 'AF-C0H3V8-F1.pdb'
output_path = './data/test'

S,H,V1,V2, df = aa_mut_select(input_path, pdb_file, output_path, df=True)


pqr_file finished
1
PQR file already exists
Prot_df finished
Salt_bridge finished
H_bond finished
VdW_interaction finished


c:\Users\marik\OneDrive - bwedu\Uni HD\FS 4\Bioinfo\topic04_02\helper_function.py:61: RuntimeWarning: invalid value encountered in cast
  distance[:,0] = distance[:,0].astype('int')


In [166]:
from scipy.sparse import lil_matrix
name = pdb_file.split('-')[1]
arr = H[name][:,:,0]

rows_with_nan = np.insert(np.array([np.all(np.isnan(arr[1:, 1:]), axis=1)]),0, None) #find rows with all nan values
cols_with_nan = np.insert(np.array([np.all(np.isnan(arr[1:, 1:]), axis=0)]),0, None) #find columns with all nan values
arr = arr[~rows_with_nan, :] #delete rows with all nan values
arr = arr[:, ~cols_with_nan] #delete columns with all nan values
arr[:,0] = arr[:,0].astype('int')

print(arr)


[[-2.14748365e+09  7.00000000e+01  1.90000000e+02]
 [ 9.22000000e+02             nan  2.06793977e+00]
 [ 1.01700000e+03  2.15041768e+00             nan]]


C:\Users\marik\AppData\Local\Temp\ipykernel_14220\4095873690.py:9: RuntimeWarning: invalid value encountered in cast
  arr[:,0] = arr[:,0].astype('int')


(27, 2)
[[False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]]


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed